In [1]:
!git clone https://github.com/jclugor/AMC.git

Cloning into 'AMC'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (419/419), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 1399 (delta 223), reused 390 (delta 223), pack-reused 980 (from 1)
Receiving objects: 100% (1399/1399), 387.41 MiB | 30.81 MiB/s, done.
Resolving deltas: 100% (386/386), done.


In [2]:
%cd 'AMC'

/kaggle/working/AMC


In [3]:
import os
import sys
import pickle
import numpy as np
import mltools
from rmldataset2016 import load_data
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [4]:
sys.path.append(os.path.abspath(f'/kaggle/working/AMC/1DCNN-PF')) # cant import from module starting with number
sys.path.append(os.path.abspath(f'/kaggle/working/AMC/PET-CGDNN'))
sys.path.append(os.path.abspath(f'/kaggle/working/AMC/IC-AMCNet'))
sys.path.append(os.path.abspath(f'/kaggle/working/AMC/TAD'))
from DCNNPF import DLmodel
from CGDNet.CGDNN import CGDNN
# from CLDNN.CLDNNLikeModel import CLDNNLikeModel
import CLDNN.CLDNNLikeModel as cldnn
import CLDNN2.CLDNNLikeModel as cldnn2
from CNN1.CNN2Model import CNN2Model
from CNN2.CNN2 import CNN2
from DAE.DAE import DAE
from DenseNet.DenseNet import DenseNet
from GRU2.GRUModel import GRUModel
from ICAMC import ICAMC
from LSTM2.CuDNNLSTMModel import LSTMModel
from MCLDNN.MCLDNN import MCLDNN
from MCNET.MCNET import MCNET
from PETCGDNN import PETCGDNN
from ResNet.ResNet import ResNet
from TAD import MCLDNN_VGN

In [5]:
models_dict = {'1DCNN-PF': DLmodel,
               'CGDNet': CGDNN,
               'CLDNN': cldnn.CLDNNLikeModel,
               'CLDNN2': cldnn2.CLDNNLikeModel,
               'CNN1': CNN2Model,
               'CNN2': CNN2,
               'DAE': DAE,
               'DenseNet': DenseNet,
               'GRU2': GRUModel,
               'IC-AMCNet': ICAMC,
               'LSTM2': LSTMModel,
               'MCLDNN': MCLDNN,
               'MCNET': MCNET,
               'PET-CGDNN': PETCGDNN,
               'ResNet': ResNet,
               'TAD': MCLDNN_VGN.MCLDNN}

In [6]:
idx = []
for i, idx_to_load in enumerate(["train_idx", "val_idx", "test_idx"]):
    with open(f'/kaggle/input/rml201610a-idx/{idx_to_load}.pkl', 'rb') as file:
        idx.append(pickle.load(file))
(mods,snrs,lbl),(X_train,Y_train),(X_val,Y_val),(X_test,Y_test),(train_idx,val_idx,test_idx) = load_data(idx=idx)

In [7]:
nb_epoch = 1000
batch_size = 512

In [8]:
# Helper functions
def rotate_matrix(theta):
    m = np.zeros((2, 2))
    m[0, 0] = np.cos(theta)
    m[0, 1] = -np.sin(theta)
    m[1, 0] = np.sin(theta)
    m[1, 1] = np.cos(theta)
    print(m)
    return m

def Rotate_DA(x, y):
    N, L, C = np.shape(x)
    x_rotate1 = np.matmul(x, rotate_matrix(np.pi/2))
    x_rotate2 = np.matmul(x, rotate_matrix(np.pi))
    x_rotate3 = np.matmul(x, rotate_matrix(3 * np.pi / 2))

    x_DA = np.vstack((x, x_rotate1, x_rotate2, x_rotate3))

    y_DA = np.tile(y, (1, 4)).T.reshape(-1)
    return x_DA, y_DA


def norm_pad_zeros(X_train,nsamples):
    print ("Pad:", X_train.shape)
    for i in range(X_train.shape[0]):
        X_train[i,:,0] = X_train[i,:,0]/np.linalg.norm(X_train[i,:,0],2)
    return X_train

def l2_normalize(x, axis=-1):
    y = np.sum(x ** 2, axis, keepdims=True)
    return x / np.sqrt(y)
    

def to_amp_phase(X_train,X_val,X_test,nsamples):
    X_train_cmplx = X_train[:,0,:] + 1j* X_train[:,1,:]
    X_val_cmplx = X_val[:,0,:] + 1j* X_val[:,1,:]
    X_test_cmplx = X_test[:,0,:] + 1j* X_test[:,1,:]
    
    X_train_amp = np.abs(X_train_cmplx)
    X_train_ang = np.arctan2(X_train[:,1,:],X_train[:,0,:])/np.pi
    
    
    X_train_amp = np.reshape(X_train_amp,(-1,1,nsamples))
    X_train_ang = np.reshape(X_train_ang,(-1,1,nsamples))
    
    X_train = np.concatenate((X_train_amp,X_train_ang), axis=1) 
    X_train = np.transpose(np.array(X_train),(0,2,1))

    X_val_amp = np.abs(X_val_cmplx)
    X_val_ang = np.arctan2(X_val[:,1,:],X_val[:,0,:])/np.pi
    
    
    X_val_amp = np.reshape(X_val_amp,(-1,1,nsamples))
    X_val_ang = np.reshape(X_val_ang,(-1,1,nsamples))
    
    X_val = np.concatenate((X_val_amp,X_val_ang), axis=1) 
    X_val = np.transpose(np.array(X_val),(0,2,1))
    
    X_test_amp = np.abs(X_test_cmplx)
    X_test_ang = np.arctan2(X_test[:,1,:],X_test[:,0,:])/np.pi
    
    
    X_test_amp = np.reshape(X_test_amp,(-1,1,nsamples))
    X_test_ang = np.reshape(X_test_ang,(-1,1,nsamples))
    
    X_test = np.concatenate((X_test_amp,X_test_ang), axis=1) 
    X_test = np.transpose(np.array(X_test),(0,2,1))
    return (X_train,X_val,X_test)

In [9]:
# Data augmentation with rotation
#X_train = X_train.copy().transpose((0, 2, 1))
#Y_train = np.argmax(Y_train, axis=1)
#X_train, Y_train = Rotate_DA(X_train, Y_train)
#X_train = X_train.transpose((0, 2, 1))
#Y_train = to_categorical(Y_train)

In [10]:
for model_, mdl in list(models_dict.items())[8:]:
    # train_data
    X_train_, X_val_, X_test_ = X_train.copy(), X_val.copy(), X_test.copy()
    if model_ in ['GRU2']:
        X_train_ = X_train_.swapaxes(2, 1)
        X_val_ = X_val_.swapaxes(2, 1)
    elif model_ in ['LSTM2']:
        X_train_, X_val_, X_test_ = to_amp_phase(X_train_, X_val_, X_test_, 128)
        X_train_ = norm_pad_zeros(X_train_[:, :128, :], 128)
        X_val_ = norm_pad_zeros(X_val_[:, :128, :], 128)
    elif model_ in ['1DCNN-PF']:
        X_train_, X_val_, X_test_ = to_amp_phase(X_train_, X_val_, X_test_, 128)

        X_train_ = norm_pad_zeros(X_train_[:, :128, :], 128)
        X_val_ = norm_pad_zeros(X_val_[:, :128, :], 128)
        
        X1_train = X_train_[:, :, 0]
        X1_val = X_val_[:, :, 0]
        X2_train = X_train_[:, :, 1]
        X2_val = X_val_[:, :, 1]
        X_train_ = [X1_train, X2_train]
        X_val_ = [X1_val, X2_val]
        
    elif model_ in ['CGDNet']:
        X_train_ = np.expand_dims(X_train_, axis=1)
        X_val_ = np.expand_dims(X_val_, axis=1)
        
    elif model_ in ['CLDNN']:
        X_train_ = np.reshape(X_train_, (-1, 1, 2, 128))
        X_val_ = np.reshape(X_val_, (-1, 1, 2, 128))
        
    elif model_ in ['CLDNN2', 'DenseNet', 'IC-AMCNet', 'ResNet', 'TAD']:
        X_train_ = np.expand_dims(X_train_, axis=3)
        X_val_ = np.expand_dims(X_val_, axis=3)
                               
    elif model_ in ['DAE']:
        X_train_, X_val_, X_test_ = to_amp_phase(X_train_, X_val_, X_test_, 128)
        X_train_[:, :, 0] = l2_normalize(X_train_[:, :, 0])
        X_val_[:, :, 0] = l2_normalize(X_val_[:, :, 0])
        for i in range(X_train_.shape[0]):
            k = 2/(X_train_[i,:,1].max() - X_train_[i,:,1].min())
        X_train_[i,:,1]=-1+k*(X_train_[i,:,1]-X_train_[i,:,1].min())
        for i in range(X_val_.shape[0]):
            k = 2/(X_val_[i,:,1].max() - X_val_[i,:,1].min())
        X_val_[i,:,1]=-1+k*(X_val_[i,:,1]-X_val_[i,:,1].min())

    elif model_ in ['MCLDNN']:
        X1_train = np.expand_dims(X_train_[:, 0, :], axis=2)
        X1_val = np.expand_dims(X_val_[:, 0, :], axis=2)
        X2_train = np.expand_dims(X_train_[:, 1, :], axis=2)
        X2_val = np.expand_dims(X_val_[:, 1, :], axis=2)
        X_train_t = np.expand_dims(X_train_, axis=3)
        X_val_t = np.expand_dims(X_val_, axis=3)        
        X_train_ = [X_train_t, X1_train, X2_train]
        X_val_ = [X_val_t, X1_val, X2_val]

    elif model_ in ['MCNET']:
        X_train_ = np.expand_dims(X_train_, axis=3)
        X_val_ = np.expand_dims(X_val_, axis=3)

    elif model_ in ['PET-CGDNN']:
        X_train_ = X_train_.swapaxes(2, 1)
        X_val_ = X_val_.swapaxes(2, 1)
        X1_train = X_train_[:, :, 0]
        X2_train = X_train_[:, :, 1]
        X1_val = X_val_[:, :, 0]
        X2_val = X_val_[:, :, 1]
        X_train_ = np.expand_dims(X_train_, axis=3)
        X_val_ = np.expand_dims(X_val_, axis=3)
        X_train_ = [X_train_, X1_train, X2_train]
        X_val_ = [X_val_, X1_val, X2_val]
    else:
        X_train_ = X_train
        X_val_ = X_val

    if model_ in ['DAE']:
        Y_train_ = [Y_train, X_train_]
        Y_val_ = [Y_val, X_val_]
    else:
        Y_train_ = Y_train
        Y_val_ = Y_val
    
    filepath = f'results/{model_}'
    os.makedirs(filepath, exist_ok=True)

    # optimization params
    if model_ in ['DAE']:
        compile_params = {'optimizer':Adam(),
                          'loss':{'xc': 'categorical_crossentropy', 'xd': 'mean_squared_error'},
                          'loss_weights':{'xc': 0.1, 'xd': 0.9},
                          'metrics':['accuracy', 'mse']}
    else:
        compile_params = {'optimizer':Adam(),
                          'loss':'categorical_crossentropy',
                          'metrics':['accuracy']}
    model = mdl()
    model.compile(**compile_params)
    callbacks=[
        ModelCheckpoint(filepath + '/weights.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, verbose=1, patience=5, min_lr=0.0000001),
        EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto', min_delta=0.00001)
    ]
    history = model.fit(X_train_, Y_train_, batch_size=batch_size, epochs=nb_epoch, verbose=0,
                        validation_data=[X_val_, Y_val_], callbacks=callbacks)
    if model_ == 'DAE':
        history.history['accuracy'] = history.history['xc_accuracy']
        history.history['val_accuracy'] = history.history['val_xc_accuracy']
    mltools.show_history(history, filepath)
    print(f'Finished training {model_}')
    
    !zip -r results.zip /kaggle/working/AMC/results


Epoch 1: val_loss improved from inf to 2.38857, saving model to results/GRU2/weights.keras

Epoch 2: val_loss improved from 2.38857 to 2.27779, saving model to results/GRU2/weights.keras

Epoch 3: val_loss improved from 2.27779 to 2.14849, saving model to results/GRU2/weights.keras

Epoch 4: val_loss improved from 2.14849 to 1.98085, saving model to results/GRU2/weights.keras

Epoch 5: val_loss improved from 1.98085 to 1.77780, saving model to results/GRU2/weights.keras

Epoch 6: val_loss improved from 1.77780 to 1.73965, saving model to results/GRU2/weights.keras

Epoch 7: val_loss improved from 1.73965 to 1.65582, saving model to results/GRU2/weights.keras

Epoch 8: val_loss improved from 1.65582 to 1.60957, saving model to results/GRU2/weights.keras

Epoch 9: val_loss improved from 1.60957 to 1.57234, saving model to results/GRU2/weights.keras

Epoch 10: val_loss improved from 1.57234 to 1.56375, saving model to results/GRU2/weights.keras

Epoch 11: val_loss improved from 1.56375 t

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: kaggle/working/AMC/results/ (stored 0%)
  adding: kaggle/working/AMC/results/GRU2/ (stored 0%)
  adding: kaggle/working/AMC/results/GRU2/train_loss.txt (deflated 58%)
  adding: kaggle/working/AMC/results/GRU2/total_loss.pdf (deflated 29%)
  adding: kaggle/working/AMC/results/GRU2/val_loss.txt (deflated 59%)
  adding: kaggle/working/AMC/results/GRU2/val_acc.txt (deflated 60%)
  adding: kaggle/working/AMC/results/GRU2/total_acc.pdf (deflated 29%)
  adding: kaggle/working/AMC/results/GRU2/train_acc.txt (deflated 57%)
  adding: kaggle/working/AMC/results/GRU2/weights.keras (deflated 8%)


I0000 00:00:1732226742.166491      76 service.cc:145] XLA service 0x7b281c0514e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732226742.166550      76 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-11-21 22:05:47.236843: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng12{k11=2} for conv (f32[512,128,1,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[512,128,1,71]{3,2,1,0}, f32[128,128,1,8]{3,2,1,0}, f32[128]{0}), window={size=1x8}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-11-21 22:05:47.405924: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.169200684s
Trying algorithm e


Epoch 1: val_loss improved from inf to 1.87616, saving model to results/IC-AMCNet/weights.keras

Epoch 2: val_loss improved from 1.87616 to 1.69200, saving model to results/IC-AMCNet/weights.keras

Epoch 3: val_loss improved from 1.69200 to 1.57491, saving model to results/IC-AMCNet/weights.keras

Epoch 4: val_loss improved from 1.57491 to 1.45684, saving model to results/IC-AMCNet/weights.keras

Epoch 5: val_loss improved from 1.45684 to 1.40180, saving model to results/IC-AMCNet/weights.keras

Epoch 6: val_loss improved from 1.40180 to 1.37083, saving model to results/IC-AMCNet/weights.keras

Epoch 7: val_loss did not improve from 1.37083

Epoch 8: val_loss improved from 1.37083 to 1.35470, saving model to results/IC-AMCNet/weights.keras

Epoch 9: val_loss did not improve from 1.35470

Epoch 10: val_loss improved from 1.35470 to 1.32043, saving model to results/IC-AMCNet/weights.keras

Epoch 11: val_loss did not improve from 1.32043

Epoch 12: val_loss improved from 1.32043 to 1.312

...


Epoch 1: val_loss improved from inf to 1.80609, saving model to results/ResNet/weights.keras

Epoch 2: val_loss improved from 1.80609 to 1.55812, saving model to results/ResNet/weights.keras

Epoch 3: val_loss improved from 1.55812 to 1.47899, saving model to results/ResNet/weights.keras

Epoch 4: val_loss improved from 1.47899 to 1.44720, saving model to results/ResNet/weights.keras

Epoch 5: val_loss improved from 1.44720 to 1.42388, saving model to results/ResNet/weights.keras

Epoch 6: val_loss improved from 1.42388 to 1.39998, saving model to results/ResNet/weights.keras

Epoch 7: val_loss improved from 1.39998 to 1.38374, saving model to results/ResNet/weights.keras

Epoch 8: val_loss improved from 1.38374 to 1.34643, saving model to results/ResNet/weights.keras

Epoch 9: val_loss did not improve from 1.34643

Epoch 10: val_loss improved from 1.34643 to 1.33805, saving model to results/ResNet/weights.keras

Epoch 11: val_loss improved from 1.33805 to 1.32825, saving model to res

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


updating: kaggle/working/AMC/results/ (stored 0%)
updating: kaggle/working/AMC/results/GRU2/ (stored 0%)
updating: kaggle/working/AMC/results/GRU2/train_loss.txt (deflated 58%)
updating: kaggle/working/AMC/results/GRU2/total_loss.pdf (deflated 29%)
updating: kaggle/working/AMC/results/GRU2/val_loss.txt (deflated 59%)
updating: kaggle/working/AMC/results/GRU2/val_acc.txt (deflated 60%)
updating: kaggle/working/AMC/results/GRU2/total_acc.pdf (deflated 29%)
updating: kaggle/working/AMC/results/GRU2/train_acc.txt (deflated 57%)
updating: kaggle/working/AMC/results/GRU2/weights.keras (deflated 8%)
updating: kaggle/working/AMC/results/IC-AMCNet/ (stored 0%)
updating: kaggle/working/AMC/results/IC-AMCNet/train_loss.txt (deflated 60%)
updating: kaggle/working/AMC/results/IC-AMCNet/total_loss.pdf (deflated 27%)
updating: kaggle/working/AMC/results/IC-AMCNet/val_loss.txt (deflated 60%)
updating: kaggle/working/AMC/results/IC-AMCNet/val_acc.txt (deflated 64%)
updating: kaggle/working/AMC/results/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


updating: kaggle/working/AMC/results/ (stored 0%)
updating: kaggle/working/AMC/results/GRU2/ (stored 0%)
updating: kaggle/working/AMC/results/GRU2/train_loss.txt (deflated 58%)
updating: kaggle/working/AMC/results/GRU2/total_loss.pdf (deflated 29%)
updating: kaggle/working/AMC/results/GRU2/val_loss.txt (deflated 59%)
updating: kaggle/working/AMC/results/GRU2/val_acc.txt (deflated 60%)
updating: kaggle/working/AMC/results/GRU2/total_acc.pdf (deflated 29%)
updating: kaggle/working/AMC/results/GRU2/train_acc.txt (deflated 57%)
updating: kaggle/working/AMC/results/GRU2/weights.keras (deflated 8%)
updating: kaggle/working/AMC/results/IC-AMCNet/ (stored 0%)
updating: kaggle/working/AMC/results/IC-AMCNet/train_loss.txt (deflated 60%)
updating: kaggle/working/AMC/results/IC-AMCNet/total_loss.pdf (deflated 27%)
updating: kaggle/working/AMC/results/IC-AMCNet/val_loss.txt (deflated 60%)
updating: kaggle/working/AMC/results/IC-AMCNet/val_acc.txt (deflated 64%)
updating: kaggle/working/AMC/results/